# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.114738e+02     1.467158e+00
 * time: 0.5215170383453369
     1     1.119371e+01     7.843527e-01
 * time: 1.1997201442718506
     2    -1.049953e+01     9.943877e-01
 * time: 1.3137290477752686
     3    -3.353798e+01     7.497639e-01
 * time: 1.394252061843872
     4    -4.707096e+01     5.824759e-01
 * time: 1.4879200458526611
     5    -5.663288e+01     2.042883e-01
 * time: 1.5908870697021484
     6    -5.959705e+01     1.746945e-01
 * time: 1.6823790073394775
     7    -6.081914e+01     6.424788e-02
 * time: 1.74436616897583
     8    -6.135447e+01     4.353276e-02
 * time: 1.806344985961914
     9    -6.165574e+01     4.538364e-02
 * time: 1.8694441318511963
    10    -6.187043e+01     2.773035e-02
 * time: 1.9320690631866455
    11    -6.203305e+01     1.843856e-02
 * time: 1.9950981140136719
    12    -6.211472e+01     1.447094e-02
 * time: 2.073742151260376
    13    -6.214947e+01     1.274872e-02
 * time: 2.135946035385132

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671083
    AtomicLocal         -18.8557696
    AtomicNonlocal      14.8522641
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485386 
    Xc                  -19.3336826

    total               -62.261666458451